In [17]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import math
import pandas as pd
import numpy as np
import random
from deap import creator, base, tools, algorithms
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold


def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

def getFitness(data,decision_attr,individual):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    feature_columns = [col for col in data.columns if col not in ['id','target']]
    selected_features = [feature_columns[i] for i in range(len(feature_columns)) if individual[i] == 1]

    equivalence_classes = calculate_equivalence_classes(data,selected_features)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            if len(x)>total_records*0.0:
                y = 1-len(x) / class_size
                total_rule+=len(x)
                rule_confidences.append(y)
            if len(x)>0:
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)
    rule_confidences = np.array(rule_confidences)
    # rule_confidences = [x for x in rule_confidences if x <= 0.6]
    count = len(rule_confidences)
    if count == 0:
        return(999,);
    # if total_rule/total_records<0.1:
    #     return (1,);

    avg_confidence = np.sum(rule_confidences) / count +wight1*(entropy_D-entropy_DB)+wight2*(count/total_records)
    return(avg_confidence,)

def geneticAlgorithm(df,y,n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    random.seed(1)
    # create individual
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness,df,y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    # stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    top_5_hof = sorted(hof.items, key=lambda x: x.fitness.values[0])[:1]
    for i, ind in enumerate(top_5_hof):
        print(f"Top {i+1} solution: {ind}\nFitness: {ind.fitness.values[0]}\n")
    return top_5_hof
    # return hof



In [18]:
def bestIndividual_KNN(hof):
    k = 10
    knn = KNeighborsClassifier(n_neighbors=k)
    for ind in hof:
        df = pd.read_csv('data/German2.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y=df["target"]
        X=df[selected_features]
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)
        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
        return mean_accuracy,selected_features

In [19]:
from sklearn.model_selection import StratifiedKFold

def bestIndividual_SVM(hof):
    def SVM(trainX, trainY, testX, testY):
        scaler = StandardScaler()
        scaler.fit(trainX)
        trainX3 = scaler.transform(trainX)
        testX3 = scaler.transform(testX)
        clf = svm.SVC(C=10, gamma=0.8, max_iter=1000)
        clf.fit(trainX3, trainY)
        test_result = clf.predict(testX3)
        precision = np.sum(test_result == testY) / testY.shape[0]
        return precision

    maxaccuracy = 0.0
    for ind in hof:
        df =pd.read_csv('data/German.csv')
        feature_columns = ['status_account', 'duration', 'credit_history', 'purpose', 'amount', 'svaing_account',
           'present_emp', 'income_rate', 'personal_status','other_debtors',
            'residence_info', 'property', 'age', 'inst_plans','housing', 'num_credits',
            'job', 'dependents', 'telephone', 'foreign_worker']
        from sklearn.preprocessing import LabelEncoder

        labelencoder=LabelEncoder()
        for col in feature_columns:
            df[col] = labelencoder.fit_transform(df[col])
        df['target'] = df['target'].map({2 : 0, 1: 1})
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        # selected_features=['status_account', 'duration', 'other_debtors', 'inst_plans', 'foreign_worker']
        Y = df["target"]
        X = df[selected_features]

        # Use stratified k-fold cross-validation
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        scores = []
        for train_index, test_index in skf.split(X, Y):
            trainX, testX = X.iloc[train_index], X.iloc[test_index]
            trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
            score = SVM(trainX, trainY, testX, testY)
            scores.append(score)
        avg_score = np.mean(scores)

        if avg_score > maxaccuracy:
            maxaccuracy = avg_score
            best_feature = selected_features

    return maxaccuracy, best_feature

In [20]:
from sklearn.tree import DecisionTreeClassifier

def bestIndividual_DT(hof):
    for ind in hof:
        df = pd.read_csv('data/German2.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        X = df[selected_features]
        Y = df['target']
        dt = DecisionTreeClassifier(max_depth=5)
        kf = KFold(n_splits=10, shuffle=True, random_state=42)

        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            dt.fit(X_train, y_train)

            y_pred = dt.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)

        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
    return mean_accuracy

In [21]:
wighta=[0,0.1,0.2,0.3,0.4,0.5]
wightb=[0,1,2,3,4,5]
SVM=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
KNN=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
DT=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
F=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]


for i_index, i in enumerate(wighta):
    for j_index, j in enumerate(wightb):
        try:
            # 你的代码
            wight1=i
            wight2=j
            n_pop=50
            n_gen=25
            # read dataframe from csv
            df = pd.read_csv('data/German2.csv')
            # encode labels column to numbers
            y = df['target']
            feature_columns = [ col for col in df.columns if col not in ['id','target']]
            X = df[feature_columns]
            individual = [1 for i in range(len(X.columns))]
            getFitness(df,'target',individual)
            print("Accuracy with all features: \t" +
                  str(getFitness(df,'target',individual)) + "\n")

            # apply genetic algorithm
            hof = geneticAlgorithm(df,'target',n_pop, n_gen)
            # apply genetic algorithm
            hof = geneticAlgorithm(df,'target',n_pop, n_gen)

            print('------------wight1--------------')
            print(wight1)
            print('------------wight2--------------')
            print(wight2)

            accuracy_SVM,individual = bestIndividual_SVM(hof,df)
            SVM[i_index][j_index] = accuracy_SVM
            print("-----------SVM---------------")
            print(accuracy_SVM, "保存至 SVM[{}][{}]".format(i_index, j_index))

            accuracy_KNN,individual = bestIndividual_KNN(hof,df)
            KNN[i_index][j_index] = accuracy_KNN
            print("-----------KNN---------------")
            print(accuracy_KNN, "保存至 KNN[{}][{}]".format(i_index, j_index))

            accuracy_DT = bestIndividual_DT(hof,df)
            DT[i_index][j_index] = accuracy_DT
            print("-----------DT---------------")
            print(accuracy_DT, "保存至 DT[{}][{}]".format(i_index, j_index))

            print('------------特征-----------------')
            print(individual)
            F[i_index][j_index] = individual
            print("特征保存至 F[{}][{}]".format(i_index, j_index))

        except Exception as e:
            print("执行过程出现错误：", e)
            continue

Accuracy with all features: 	(0.0,)

gen	nevals	avg      	min     
0  	50    	0.0445824	0.001002
1  	27    	0.0176146	0.001002
2  	31    	0.00724676	0       
3  	25    	0.00196647	0       
4  	34    	0.00118875	0       
5  	30    	0.000503509	0       
6  	28    	0.000639701	0       
7  	40    	0.000587261	0       
8  	32    	0.000201372	0       
9  	34    	0.000181006	0       
10 	29    	0.000120746	0       
11 	28    	0.000427576	0       
12 	30    	0.000161698	0       
13 	25    	0.000140825	0       
14 	36    	0.000181353	0       
15 	32    	0.000343039	0       
16 	35    	0.000241798	0       
17 	37    	0.000342977	0       
18 	31    	0.000160824	0       
19 	33    	6.03622e-05	0       
20 	32    	0.000221656	0       
21 	31    	0.00014111 	0       
22 	35    	0.000120887	0       
23 	36    	0.000120765	0       
24 	36    	0.000100748	0       
25 	27    	0.000201473	0       
Top 1 solution: [1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1]
Fitness: 0.0

gen	nevals	avg   